In [20]:
import os 
import numpy as np
import pandas as pd
from typing  import List
from sklearn.datasets import load_iris

ROOT= "iris_course"
ARTIFACTS = os.path.join(ROOT,'artifacts')
os.makedirs(ARTIFACTS,exist_ok=True)

# axis=0 與 axis=1 差別
# print("axis=0：每欄平均", X.mean(axis=0))
# print("axis=1：每列平均", X.mean(axis=1))

# 這段的作用
# 這段是 把特徵名稱 (names)、平均值 (m)、標準差 (s) 串在一起逐一輸出。

# 📌 詳細拆解
# zip(names, m, s)
# names 是特徵名稱的字串清單，例如：
# ["sepal_length","sepal_width","petal_length","petal_width"]

# m 是各欄的平均值陣列，例如：
# [5.84, 3.05, 3.76, 1.20]

# s 是各欄的標準差陣列，例如：
# [0.82, 0.43, 1.76, 0.76]

# zip() 會把它們一一對應打包成 tuple：
# [
#   ("sepal_length", 5.84, 0.82),
#   ("sepal_width",  3.05, 0.43),
#   ...
# ]

# 格式	     說明
# {n:<14s}	左對齊字串，寬度 14
# {mi:8.4f}	浮點數，總寬度 8，顯示 4 位小數
# {sd:8.4f}	同上

def describe_stats(X:np.ndarray,names:List[str],title:str):
    m,s = X.mean(axis=0), X.std(axis=0)
    print(f"\n[{title}]")
    for n, mi, sd in zip(names, m,s):
        print(f"{n:<14s} mean={mi:8.4f} std={sd:8.4f}" )

print("***Step1 載入資料與探索")
iris =load_iris()
# print(iris)


# 載入資料集：
# x 是 (150,4) 的數值矩陣
# y 是 (150,) 的標籤（0,1,2）
# 這四個特徵分別是：花萼長寬、花瓣長寬
x,y = iris.data,iris.target

# sepal 萼片
# petal 花瓣
feature_names = ["sepal_length","sepal_width","petal_length","petal_width"]
target_names=iris.target_names.tolist()
# print(target_names)

# 這行會用 pandas 把 x 轉成一個 DataFrame（表格格式），欄名就是特徵名稱：
df=pd.DataFrame(x,columns=feature_names)
df["target"] = y
print("\n 前5筆資料");print(df.head())
print("\n 類別分布:")


# enumerate() 是 Python 內建函式
# 它的作用是：在迴圈中同時取得「索引」和「元素」

# 小總結
#表達式	                           意思	結果型態
# y	類別標籤陣列	           ndarray of int
# y == i	          元素逐一是否等於 i	ndarray of bool
# (y == i).sum()	      有幾個等於 i	int（計數結果）

# ✅ 所以雖然 y 是個整數陣列 (ndarray)，
# 用 == 去比對數值時，會自動對每個元素做比較，這就是 NumPy 的「向量化運算」。

# target_names：只會是 三個獨特的名稱 → ['setosa','versicolor','virginica']
for i ,name in enumerate(target_names):
    print(f"{i}={name:<10s} : {(y==i).sum()} 筆")
describe_stats(x,feature_names,"原始資料(未標準化)")

out_csv=os.path.join(ARTIFACTS,"iris_preview.csv")
# index=False 表示 不要輸出 DataFrame 的索引欄位（只保留資料本身）
df.head(20).to_csv(out_csv,index=False)

print(f"\n->已存取20筆預覽 :{out_csv}")
print("STEP1 完成")

***Step1 載入資料與探索

 前5筆資料
   sepal_length  sepal_width  petal_length  petal_width  target
0           5.1          3.5           1.4          0.2       0
1           4.9          3.0           1.4          0.2       0
2           4.7          3.2           1.3          0.2       0
3           4.6          3.1           1.5          0.2       0
4           5.0          3.6           1.4          0.2       0

 類別分布:
0=setosa     : 50 筆
1=versicolor : 50 筆
2=virginica  : 50 筆

[原始資料(未標準化)]
sepal_length   mean=  5.8433 std=  0.8253
sepal_width    mean=  3.0573 std=  0.4344
petal_length   mean=  3.7580 std=  1.7594
petal_width    mean=  1.1993 std=  0.7597

->已存取20筆預覽 :iris_course\artifacts\iris_preview.csv
STEP1 完成


In [21]:
from sklearn.model_selection import train_test_split
print("\n===STEP2 | 切分 Train/Val/Test===")



# x：所有的特徵資料（numpy.ndarray，shape = (150, 4)）
# y：所有的標籤（numpy.ndarray，shape = (150,)）
# train_test_split 是 scikit-learn 提供的
# train_test_split 函式，用來隨機把資料拆成兩組
# 參數解釋
# 參數	                            用途
# x, y	             輸入資料與標籤
# test_size=0.2	    指定 20% 當「測試集」，剩下 80% 是「訓練+驗證」
# random_state=42	設定隨機種子，讓結果可重現
# stratify(分層)=y	    分層抽樣：確保三個類別在兩組中比例一致

X_trainval,X_test,y_trainval,y_test =train_test_split(
    x,y,test_size=0.2,random_state=42,stratify=y
)
X_train,X_val,y_train,y_val =train_test_split(
    X_trainval,y_trainval,test_size=0.2,random_state=42,stratify=y_trainval
)

print(f"切分形狀: train={X_train.shape} val={X_val.shape} test={X_test.shape}")
print("STEP2 完成")


===STEP2 | 切分 Train/Val/Test===
切分形狀: train=(96, 4) val=(24, 4) test=(30, 4)
STEP2 完成


In [22]:
from sklearn.preprocessing import StandardScaler
import joblib

print('\====STEP 3 | 標準化(只用訓練集fit)並存檔)')

# StandardScaler 是一個「轉換器 (transformer)」物件
# 它會計算：
# 每個欄位的平均值 μ
# 每個欄位的標準差 σ
# 然後把資料套用公式：

# 𝑧 = (x - 𝜇) / 𝜎
# 讓 每個特徵（欄）都變成平均 0、標準差 1	​


# 第 1 行：先用訓練資料「學習平均與標準差」
# .fit(X_train) 會計算：
# 每一欄的平均值 mean_
# 每一欄的標準差 scale_
# 這一步「只用訓練集」是為了避免資料洩漏（不能偷看驗證或測試資料）
scaler = StandardScaler().fit(X_train)

# 第 2 行：把訓練資料做標準化
# 用剛剛算出的 mean_ 和 scale_ 把資料轉換成：
# (原值 - 平均) / 標準差
# 結果：每一欄的平均會變成 0、標準差變成 1
X_train_sc = scaler.transform(X_train)

# 第 3～4 行：用同一個 scaler 處理驗證與測試資料
# 這裡 不能再 fit 一次，要用 訓練集的平均與標準差 來轉換
# 這樣才確保模型在驗證/測試時使用完全相同的尺度
X_val_sc = scaler.transform(X_val)
X_test_sc = scaler.transform(X_test)

describe_stats(X_train, feature_names,"訓練集(標準化前)")
describe_stats(X_train_sc, feature_names,"訓練集(標準化後)")

npz_path= os.path.join(ARTIFACTS,"train_val_test_scaled.npz")
# .npz 就是 把很多 NumPy 陣列一起打包壓縮存檔
# → 讓你之後可以 一次存、一包讀，很方便。

# 這行會建立一個 train_val_test_scaled.npz 檔，裡面包含：

# 存進去的名稱	內容
# X_train_sc	標準化後的訓練特徵資料
# y_train	訓練標籤
# X_val_sc	標準化後的驗證特徵資料
# y_val	驗證標籤
# X_test_sc	標準化後的測試特徵資料
# y_test	測試標籤
# feature_names	特徵名稱清單（轉成陣列存）
# target_names	類別名稱清單（轉成陣列存）

np.savez(npz_path,
         X_train_sc=X_train_sc,y_train=y_train,
         X_val_sc=X_val_sc,y_val=y_val,
         X_test_sc=X_test_sc,y_test=y_test,
         feature_names=np.array(feature_names,dtype=object),
         target_names=np.array(target_names,dtype=object),
)

# 這兩行的目的

# 把你訓練好的 StandardScaler 物件
# 存成一個檔案（scaler.pkl），
# 以後要用時可以直接載回來，不用重新 .fit() 一次。

# 把物件存起來
# joblib.dump(scaler, scaler_path)
# 使用 joblib 的 dump 函式
# 把 scaler（也就是你用 X_train .fit() 過的 StandardScaler）存成 .pkl 檔案
# .pkl 是「pickle」格式，用來存整個 Python 物件

scaler_path =os.path.join(ARTIFACTS,"scaler.pkl")
joblib.dump(scaler,scaler_path)

print(f"->已存標準化資料:{npz_path}")
print(f"->已存標準化器:{scaler_path}")
print("STEP3 完成")

\====STEP 3 | 標準化(只用訓練集fit)並存檔)

[訓練集(標準化前)]
sepal_length   mean=  5.8333 std=  0.8516
sepal_width    mean=  3.0083 std=  0.4264
petal_length   mean=  3.7500 std=  1.7530
petal_width    mean=  1.1875 std=  0.7463

[訓練集(標準化後)]
sepal_length   mean=  0.0000 std=  1.0000
sepal_width    mean=  0.0000 std=  1.0000
petal_length   mean=  0.0000 std=  1.0000
petal_width    mean=  0.0000 std=  1.0000
->已存標準化資料:iris_course\artifacts\train_val_test_scaled.npz
->已存標準化器:iris_course\artifacts\scaler.pkl
STEP3 完成


In [23]:
import torch
from torch.utils.data import TensorDataset,DataLoader
# 整體目標
# 這段是 STEP4：把資料轉成 torch.Tensor，再包進 DataLoader，用來訓練模型
# 是 PyTorch 的標準資料處理流程。
# 簡單來說：
# 把資料(numpy.ndarray) → 轉成 Tensor → 用 DataLoader 分成小批次（batch） → 之後可以丟給模型訓練
print('\====STEP 4 | Tensor 與 DataLoader')


X_train_t = torch.tensor(X_train_sc,dtype=torch.float32)
y_train_t = torch.tensor(y_train,dtype=torch.long)
X_val_t = torch.tensor(X_val_sc,dtype=torch.float32)
y_val_t = torch.tensor(y_val,dtype=torch.long)


# TensorDataset(X, y)：把 X 和 y 包成一筆一筆的資料
# DataLoader(..., batch_size=16)：每次會吐出 16 筆資料（小批次）
# shuffle=True：訓練集會隨機打亂順序（避免模型記住順序）
# shuffle=False：驗證集保持原順序
# 為什麼驗證/測試集不要 shuffle
# 驗證時只是「評估」模型表現，不需要也不應打亂
# 保持固定順序 → 方便對照預測與真實標籤
# 每次評估結果一致，避免隨機性影響評估
train_loader =DataLoader(TensorDataset(X_train_t,y_train_t),batch_size=16,shuffle=True)
val_loader =DataLoader(TensorDataset(X_val_t,y_val_t),batch_size=16,shuffle=False)


# 目的

# 從 DataLoader（PyTorch）
# 取出「第一個 batch（小批次）」的資料，
# 用來 檢查資料長相是否正確。


# train_loader 是你剛建立的 DataLoader，裡面有所有訓練資料（已分好 batch）
# iter(train_loader) → 建立一個「迭代器」
# next(...) → 從迭代器中取出第一組 (特徵, 標籤)
# 結果會是：
# xb = 一個 batch 的特徵資料
# shape 通常是 (batch_size, 特徵數)
# 例如 (16, 4)
# yb = 一個 batch 的標籤資料
# shape 是 (batch_size,)
# 例如 (16,)

# 重點觀念
# shuffle=True 的 打亂時機不是在你建立 DataLoader 的當下，
# 而是在你第一次呼叫 iter(train_loader)（開始一個 epoch）時才打亂資料順序。

xb,yb = next(iter(train_loader))
print(f"第一個 batch:xb.shape={xb.shape}, yb.shape={yb.shape}")

# 取出 batch 裡第一筆資料的標籤
print(f"   xb[0](標準化後)={xb[0].tolist()}")
print(f"   yb[0](類別)={yb[0].item()}")


# 功能（一句話）
# 把你剛從 DataLoader 取出的那一個 batch（小批次）
# 轉成表格（pandas.DataFrame），加上標籤欄位，
# 再存成 CSV 檔，方便你用 Excel 或其他工具檢查。
batch_preview=os.path.join(ARTIFACTS,"batch_preview.csv")
pd.DataFrame(xb.numpy(),columns=feature_names).assign(label=yb.numpy()).to_csv(batch_preview,index=False)
print(f"->已存batch 預覽{batch_preview}")
print("STEP4 完成")

\====STEP 4 | Tensor 與 DataLoader
第一個 batch:xb.shape=torch.Size([16, 4]), yb.shape=torch.Size([16])
   xb[0](標準化後)=[-1.0960394144058228, -1.1921887397766113, 0.42784440517425537, 0.6867437362670898]
   yb[0](類別)=2
->已存batch 預覽iris_course\artifacts\batch_preview.csv
STEP4 完成


In [24]:
from torch import nn 
MODELS = os.path.join(ROOT,"models")
os.makedirs(MODELS,exist_ok=True)

print('\====STEP 5 | 定義模型與參數量')

class IrisMLP(nn.Module):
    def __init__(self, in_dim=4, hidden1=64,hidden2=32,out_dim=3,dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim,hidden1),nn.ReLU(),nn.Dropout(dropout),
            nn.Linear(hidden1,hidden2),nn.ReLU(),nn.Dropout(dropout),
            nn.Linear(hidden2,out_dim)
        )


    #     forward：定義前向傳遞
    # def forward(self, x):
    #     return self.net(x)


    # 告訴 PyTorch 資料要怎麼經過網路

    # 只要呼叫 model(x)，就會自動跑這段
    def forward(self,x): return self.net(x)




# 計算 PyTorch 模型中「可訓練參數」的總數
# 也就是：這個模型裡 所有需要更新的權重參數 一共有幾個數值（weights / biases）。

# 你問的 -> int 是 Type Hint（型別註解） 的一種，
# 不是程式功能的一部分，只是「告訴人或工具：這個函式會回傳什麼型別」。
def count_trainable_params(model: nn.Module) -> int:

    #     p.numel()
    # 回傳這個 Tensor 裡「有幾個元素」
    # 例如：
    # p.shape = (64, 4) → p.numel() = 256
    # p.shape = (64,)   → p.numel() = 64

    #     (p.numel() for p in ...) 是 生成器，不是陣列，
    # 它會「一個一個產生數字」，讓 sum() 去加總，
    # 而不會先把所有數字存在記憶體裡。

    # requires_grad 是什麼
    # 它的意思是：
    # 這個張量是否要在反向傳播（backpropagation）時計算梯度
    # 有些參數可能：
    # 是凍結的（不想訓練）
    # 是固定的 embedding 或預訓練權重
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



# 檢查你的電腦有沒有 NVIDIA GPU（CUDA）可以用，
# 然後設定一個 torch.device 物件，
# 讓你之後可以把模型或資料移到 GPU 或 CPU 執行。
# device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# 建立模型物件
# 把模型搬到指定的裝置（CPU / GPU）上
model = IrisMLP().to(device)
print(model)
print(f"可訓練參數量:{count_trainable_params(model):,}")

arch_txt =os.path.join(MODELS,"model_arch.txt")

# 功能總覽

# 把 模型的結構 和 可訓練參數總數
# 寫進一個文字檔（arch_txt）

# 用 Python 內建的檔案操作
# "w" → 以「寫入模式」開啟檔案
# encoding="utf-8" → 用 UTF-8 編碼（可以正常寫中文）
# as f → 建立檔案物件 f
# with 會在寫完後自動關閉檔案

with open(arch_txt,"w",encoding="utf-8") as f:
    f.write(str(model) + "\n")
    f.write(f"trainable_params={count_trainable_params(model)}\n")
print(f"-> 已存結構描述:{arch_txt}")
print("STEP5 完成") 

\====STEP 5 | 定義模型與參數量
IrisMLP(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=32, out_features=3, bias=True)
  )
)
可訓練參數量:2,499
-> 已存結構描述:iris_course\models\model_arch.txt
STEP5 完成


In [25]:
print("\n=== STEP 6 訓練 (含早停) ===")

# 這是定義「損失函式 (Loss Function)」的部分
criterion = nn.CrossEntropyLoss()
# 這是定義「優化器 (Optimizer)」，也就是更新模型參數的演算法。
# Adam：一種改良版的 SGD，會根據歷史梯度的大小自動調整學習率 (learning rate)，收斂速度通常比單純的 SGD 快。
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def evaluate(m, loader):

    # 1️⃣ m.eval()
    # 把模型切換到「推論模式」。
    # 會停用 Dropout、BatchNorm 等僅訓練用的機制，確保驗證結果穩定。
    m.eval()

    total, correct, loss_sum = 0, 0, 0.0

    # 2️⃣ torch.no_grad()
    # 在這個區塊內不會記錄梯度，節省記憶體和運算量。
    # 驗證或推論時不需要反向傳播，所以關掉 gradient tracking。
    with torch.no_grad():
        # 3️⃣ for xb, yb in loader
        # 從 DataLoader 一批一批讀資料。
        # xb = 輸入 (features)，yb = 標籤 (labels)。
        for xb, yb in loader:
            # 4️⃣ xb, yb = xb.to(device), yb.to(device)
            # 把資料搬到 device（可能是 GPU 或 CPU）。
            # 確保資料和模型在同一裝置上，不然會報錯。
            xb, yb = xb.to(device), yb.to(device)

            #5️⃣ logits = m(xb)
            #m → 你的模型 (例如一個 nn.Module 定義的神經網路)。
            # xb → 一個 batch 的輸入資料 (features)，通常 shape = [batch_size, in_dim]。
            # logits = 模型輸出的「原始分數」(linear output)，還沒有經過 softmax 轉換成機率。
            # 前向傳播 (forward pass)，輸出 logits（未經 softmax 的分數）。
            # shape 通常是 [batch_size, num_classes]。
            logits = m(xb)

            print('logits',logits)
            print('yb',yb)

            # 7️⃣ 損失計算
            # loss_sum += criterion(logits, yb).item() * xb.size(0)
            # criterion(logits, yb) = 算這個 batch 的平均 loss。
            # .item() 取出 Python float。
            # 乘上 xb.size(0)（batch size），轉成「總 loss」，方便最後做加權平均。
            loss_sum += criterion(logits, yb).item() * xb.size(0)

            # 8️⃣ 正確率計算
            # correct += (logits.argmax(1) == yb).sum().item()
            # argmax = argument of the maximum
            # 就是「找出一個序列裡 最大值的位置 (索引)」。
            # logits.argmax(1) → 找出每一筆資料預測的類別索引。
            # == yb → 和真實標籤比對，回傳 True/False tensor。
            # .sum() → 計算這個 batch 預測正確的數量。
            correct += (logits.argmax(1) == yb).sum().item()

            #9️⃣ 累積樣本數
            # total += yb.size(0)
            # 紀錄一共看過多少筆樣本。
            # 最後才能算平均 loss 和正確率。
            total += yb.size(0)
    return loss_sum / total, correct / total


# 各變數的用途：
# best_state = None
# 用來存模型目前「最佳狀態」(通常是 state_dict() 的複本)。
# 剛開始還沒有最佳模型，所以是 None。

# best_val = -1.0
# 用來記錄「驗證集 (validation) 的最佳表現」
# 設成 -1.0 是因為我們希望後面第一次驗證結果一定會比它好（假設準確率 ≥ 0）。

# patience = 15
# 代表「容忍連續多少次表現沒有進步」
# 如果超過 15 次 epoch 都沒有改善，就觸發 early stopping，停止訓練。

# bad = 0
# 記錄「已經連續幾次沒有進步」
# 每當 val_acc 沒有變好，就 bad += 1；有變好就 bad = 0。
best_state, best_val, patience, bad = None, -1.0, 15, 0
hist = {"tr_loss": [], "tr_acc": [], "va_loss": [], "va_acc": []}





=== STEP 6 訓練 (含早停) ===


In [26]:
# 訓練迴圈
# for ep in range(1, 201):
# 訓練最多 200 個 epoch。
# 如果提早 early stopping，就會 break。
for ep in range(1, 201):
    # 🏋️‍♂️ 模型訓練 (train mode)
    # model.train()
    # total, correct, loss_sum = 0, 0, 0.0
    # 切換到「訓練模式」(啟用 dropout、BN 等)。
    # total：累計訓練樣本數
    # correct：累計預測正確數
    # loss_sum：累計總 loss
    model.train()
    total, correct, loss_sum = 0, 0, 0.0

    # 一個 batch 的訓練
    # for xb, yb in train_loader:
    #     xb, yb = xb.to(device), yb.to(device)
    #     logits = model(xb)
    #     loss = criterion(logits, yb)

    # 取一個 batch (xb, yb)
    # 丟進 GPU/CPU
    # 前向傳播 → 得到 logits (分數)
    # 計算 loss (CrossEntropyLoss)
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)

        #⬅️ 反向傳播 + 參數更新
        # optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()

        # zero_grad()：清掉上一輪的梯度
        # loss.backward()：反向傳播，算出梯度
        # step()：用 optimizer (Adam) 更新參數
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.item() * xb.size(0)
        correct += (logits.argmax(1) == yb).sum().item()
        total += yb.size(0)
    
    tr_loss, tr_acc = loss_sum / total, correct / total

    # 驗證 (validation)
    # va_loss, va_acc = evaluate(model, val_loader)
    # 用剛才寫的 evaluate 函數算 validation loss 和 accuracy。
    va_loss, va_acc = evaluate(model, val_loader)
    
    hist["tr_loss"].append(tr_loss)
    hist["tr_acc"].append(tr_acc)
    hist["va_loss"].append(va_loss)
    hist["va_acc"].append(va_acc)
    
    print(f"Epoch {ep:03d} | train_loss={tr_loss:.4f} acc={tr_acc:.3f} | val_loss={va_loss:.4f} acc={va_acc:.3f}")


    # Early Stopping
    # 解釋：
    # 如果 validation accuracy 變好 → 更新 best_val，存下模型狀態 best_state，並 reset bad = 0。
    # 如果沒有進步 → bad += 1
    # 如果連續 patience (15) 次都沒進步 → 早停 (early stopping)，直接結束訓練。
    #這一行：
    # {k: v.cpu() for k, v in model.state_dict().items()}
    # 等價於：
    # 取出模型所有參數 (state_dict)
    # 把每個參數從 GPU 搬到 CPU
    # 存成一個新的字典
    # 這個新字典就被存在 best_state 裡，用來保存「最佳模型」的參數。
    if va_acc > best_val:
     best_val, best_state,bad = va_acc, {k: v.cpu() for k, v in model.state_dict().items()},0
    else:
        bad += 1
        if bad >= patience:
            print(f"早停：{patience} epochs 未提升")
            break


# 載回最佳
if best_state is not None:
    model.load_state_dict(best_state)


logits tensor([[-0.3058, -0.0048,  0.1059],
        [-0.1441,  0.0519, -0.0442],
        [-0.1442,  0.0864, -0.0293],
        [-0.2625, -0.0158,  0.0164],
        [ 0.3275, -0.0046, -0.1956],
        [-0.3512, -0.0322,  0.1340],
        [ 0.3038, -0.0171, -0.1763],
        [-0.2666,  0.0081,  0.0818],
        [ 0.1845,  0.0703, -0.1612],
        [-0.1050,  0.0419, -0.0637],
        [-0.0844,  0.0860, -0.1062],
        [-0.1384,  0.0228, -0.0238],
        [ 0.2918, -0.0391, -0.1539],
        [-0.5315, -0.0209, -0.0087],
        [-0.0315,  0.1546, -0.0969],
        [-0.4474, -0.0102, -0.0110]])
yb tensor([2, 1, 2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 0, 2, 1, 2])
logits tensor([[ 0.2448,  0.0699, -0.1733],
        [ 0.2152,  0.0338, -0.1785],
        [-0.2530,  0.0175,  0.0403],
        [-0.2758, -0.0062,  0.0820],
        [-0.1488,  0.0265, -0.0156],
        [ 0.2631,  0.0658, -0.1719],
        [ 0.2426,  0.0263, -0.1847],
        [-0.0220,  0.1573, -0.0973]])
yb tensor([0, 0, 2, 2, 1, 0, 0, 1])


In [31]:
import matplotlib.pyplot as plt


# === 畫訓練/驗證曲線 ===
xs = np.arange(1, len(hist["tr_loss"]) + 1)
plt.figure(figsize=(8, 4))

# loss 曲線
plt.plot(xs, hist["tr_loss"], label="train_loss")
plt.plot(xs, hist["va_loss"], label="val_loss")

# acc 曲線
plt.plot(xs, hist["tr_acc"], label="train_acc")
plt.plot(xs, hist["va_acc"], label="val_acc")

plt.xlabel("epoch")
plt.ylabel("value")
plt.title("Training Curves")
plt.legend()
plt.tight_layout()

PLOTS = os.path.join(ROOT, "plots")

# 確保資料夾存在
os.makedirs(PLOTS, exist_ok=True)

# 儲存圖表
curve_path = os.path.join(PLOTS, "curves.png")
plt.savefig(curve_path, dpi=150)
plt.close()
print(f"✅ 已存訓練曲線: {curve_path}")

# === 儲存最佳模型權重 ===
best_path = os.path.join(MODELS, "best.pt")
torch.save(model.state_dict(), best_path)
print(f"✅ 已存最佳權重: {best_path}")

print("STEP 6 ✅ 完成")

✅ 已存訓練曲線: iris_course\plots\curves.png
✅ 已存最佳權重: iris_course\models\best.pt
STEP 6 ✅ 完成


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("\n=== STEP 7 測試集評估 ===")

# 把模型切換到 評估模式 (evaluation mode)。
# 關掉 dropout、batch normalization 的隨機行為，讓測試結果穩定。
# 模型訓練好 不會自己關掉 Dropout/BN，因為 PyTorch 不知道你要做「驗證」還是「繼續訓練」。
model.eval()

# 關閉梯度計算，節省記憶體和運算量。
# 測試時只要 forward，不需要 backward。
with torch.no_grad():

    # X_test_sc → 測試集的輸入特徵 (通常是標準化後的資料)。
    # torch.tensor(..., dtype=torch.float32) → 把 numpy array 轉成 PyTorch tensor，並指定浮點數型別。
    # .to(device) → 把資料搬到跟模型相同的裝置（CPU 或 GPU）。
    # model(...) → 前向傳播，得到 logits（未經 softmax 的分數）。
    # 假設測試集有 100 筆資料，類別數 = 3，
    # 那 logits.shape = [100, 3]。
    logits = model(torch.tensor(X_test_sc, dtype=torch.float32).to(device))


    # logits.argmax(1) → 沿著類別維度找最大值的索引，也就是「模型預測的類別」。
    # e.g. [2.0, 1.0, -0.5] → argmax = 0 (預測 class 0)
    # .cpu() → 把結果移回 CPU（因為有可能在 GPU 上運算）。
    # .numpy() → 轉成 numpy array，方便後續用 sklearn.metrics 計算 accuracy、混淆矩陣等。
    # 最後 y_pred 是一個長度 = 測試資料筆數的 array，例如：
    # array([0, 2, 1, 0, 1, 2, ...])
    y_pred = logits.argmax(1).cpu().numpy()

# 計算準確率
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy = {acc:.3f}\n")

# 分類報告
print("分類報告：")
print(classification_report(y_test, y_pred, target_names=target_names, digits=3))
    
# === 混淆矩陣 ===
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4.5, 4))
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix")
plt.colorbar()

ticks = np.arange(len(target_names))
plt.xticks(ticks, target_names, rotation=30)
plt.yticks(ticks, target_names)

# 在矩陣格子中填上數字
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, str(cm[i, j]), ha="center", va="center")

plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()

cm_path = os.path.join(PLOTS, "confusion_matrix.png")
plt.savefig(cm_path, dpi=150)
plt.close()
print(f"->已存混淆矩陣: {cm_path}")


=== STEP 7 測試集評估 ===
Test Accuracy = 0.800

分類報告：
              precision    recall  f1-score   support

      setosa      1.000     1.000     1.000        10
  versicolor      1.000     0.400     0.571        10
   virginica      0.625     1.000     0.769        10

    accuracy                          0.800        30
   macro avg      0.875     0.800     0.780        30
weighted avg      0.875     0.800     0.780        30

->已存混淆矩陣: iris_course\plots\confusion_matrix.png


In [38]:
# === 回到原單位顯示幾筆 ===

# X_test_sc → 之前經過標準化 (scaler) 的測試集特徵。
# scaler.inverse_transform(...) → 把標準化後的資料還原成原本的數值單位。
# 例如：花瓣長度原本是 3.5 cm，標準化後可能是 -0.23，現在還原回 3.5。
X_test_orig = scaler.inverse_transform(X_test_sc)

# 最多顯示 10 筆樣本。
# 如果測試資料少於 10，就顯示全部。
show_n = min(10, len(X_test_orig))

tbl = pd.DataFrame(X_test_orig[:show_n], columns=feature_names)
tbl["True"]  = [target_names[i] for i in y_test[:show_n]]
tbl["Pred"]  = [target_names[i] for i in y_pred[:show_n]]
print(tbl)
samples_csv = os.path.join(ARTIFACTS, "test_samples.csv")
tbl.to_csv(samples_csv, index=False)
print(f"已存樣本對照(原單位): {samples_csv}")

print("STEP 7 完成 ✅ （全流程）")

   sepal_length  sepal_width  petal_length  petal_width        True  \
0           4.4          3.0           1.3          0.2      setosa   
1           6.1          3.0           4.9          1.8   virginica   
2           4.9          2.4           3.3          1.0  versicolor   
3           5.0          2.3           3.3          1.0  versicolor   
4           4.4          3.2           1.3          0.2      setosa   
5           6.3          3.3           4.7          1.6  versicolor   
6           4.6          3.6           1.0          0.2      setosa   
7           5.4          3.4           1.7          0.2      setosa   
8           6.5          3.0           5.2          2.0   virginica   
9           5.4          3.0           4.5          1.5  versicolor   

         Pred  
0      setosa  
1   virginica  
2  versicolor  
3  versicolor  
4      setosa  
5   virginica  
6      setosa  
7      setosa  
8   virginica  
9   virginica  
已存樣本對照(原單位): iris_course\artifacts\test_sa